In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
cd

/Users/keith_tetrad


In [3]:
codon_variant = pd.read_csv('kw_dms/results/variants/codon_variants.csv', usecols=['barcode','library','barcode','codon_substitutions','aa_substitutions','n_codon_substitutions','n_aa_substitutions'])
sample_Both_B2T1 = pd.read_csv('kw_dms/old_results/barcode_counts/LibB-231017-Both_bio2-1_counts.csv')
sample_DMSO_B2T1 = pd.read_csv('kw_dms/old_results/barcode_counts/LibB-231017-DMSO_bio2-1_counts.csv')
BD_calc = pd.read_csv('kw_dms/old_results/func_scores/Both_B2T1-vs-B2T1_func_scores.csv')

In [4]:
def calculate_functional_score(pre_selection, post_selection, codon_variant, library, count_threshold):
    # Rename count columns
    pre_selection.rename(columns={'count': 'count_pre'}, inplace=True)
    post_selection.rename(columns={'count': 'count_post'}, inplace=True)
    
    # Filter out barcodes in pre_selection that dont meet the count and % threshold
    # total_pre_counts = pre_selection['count_pre'].sum()
#     pre_selection = pre_selection[(pre_selection['count_pre'] >= count_threshold) & 
#                                   (pre_selection['count_pre'] / total_pre_counts >= 0.0000001)]   
    
    pre_selection = pre_selection[(pre_selection['count_pre'] >= count_threshold)] 
    
    # Merge codon_variants with 
    pre_selection = pd.merge(pre_selection, codon_variant, on='barcode')
    post_selection = pd.merge(post_selection, codon_variant, on='barcode')
    
    #filter out barcodes not in intended library
    pre_selection = pre_selection[pre_selection['library'] == library]
    post_selection = post_selection[post_selection['library'] == library]
    
    # Merge pre and post counts
    merged_df = pd.merge(pre_selection, post_selection[['barcode', 'count_post']], on='barcode', how='left')
    merged_df.fillna(0, inplace=True)
    
    # Calculate total wt counts then filter them out of the df
    wt_barcodes = merged_df[merged_df['n_codon_substitutions'] == 0]
    pre_wt_counts = wt_barcodes['count_pre'].sum()
    post_wt_counts = wt_barcodes['count_post'].sum()
    merged_df = merged_df[merged_df['n_codon_substitutions'] > 0]
    
    # Change zeroes in 'count_post' to 0.5
    # Convert 'count_post' column to float
    merged_df['count_post'] = merged_df['count_post'].astype(float)

    # Now change zeroes in 'count_post' to 0.5
    merged_df.loc[merged_df['count_post'] == 0, 'count_post'] = 0.5
    
    # Calculate the functional score for each row
    merged_df['func_score'] = np.log2((merged_df['count_post'] / post_wt_counts) / (merged_df['count_pre'] / pre_wt_counts))
    merged_df = merged_df.sort_values(by='barcode', ascending=True)
    merged_df = merged_df[['barcode','func_score']]
    
    return merged_df


In [6]:
my_calc = calculate_functional_score(sample_DMSO_B2T1,sample_Both_B2T1,codon_variant,'LibB',25)
my_calc.describe()

,func_score
count,57110.000000
mean,-1.600977
std,2.976251
min,-9.709169
25%,-4.357798
50%,-0.455291
75%,0.550465
max,5.234078


In [7]:
BD_calc = BD_calc[['barcode','func_score']]
BD_calc.describe()

,func_score
count,68496.00000
mean,-1.50913
std,2.79960
min,-9.78700
25%,-3.76200
50%,-0.46960
75%,0.46520
max,4.27300
